# Part 1: Bayesian Approach

Is the one who earns more (>50K) necessarily in the different age category from the one who earns less (<=50)?

X- age Y-income

#### 1.

In [5]:
# Deprecation of jupyter warnings about sns functions that will be deprecated in the future. (sns.distplot)
import warnings
warnings.filterwarnings('ignore')

#Reading a CSV file into pandas Dataframe
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt
import random
from scipy.stats import f
from scipy.stats import norm
import pprint
import sys

random.seed(1)
# Removing missing values
missing_values = ["n/a", "na", "--","?"]
df_full = pd.read_csv("adult.csv",sep=",", na_values = missing_values)
df_full=df_full.dropna()

# Converting string columns to binary.
df_full['gender'] =df_full['gender'].map({'Female': 0, 'Male': 1})
df_full['income'] = df_full['income'].map({'>50K': 1, '<=50K': 0})

df_200=df_full.sample(n = 200)
df_without_sub_sample=df_full.drop(df_200.index)
df_1000=df_without_sub_sample.sample(n = 1000)

#### 2.

In [6]:
# Define binary indicators Z_i
def get_indicator(x,threshold):
    if x>threshold:
        return 1
    return 0

threshold_of_viewed_data=df_200['age'].median()
df_200['Z']=[get_indicator(x,threshold_of_viewed_data) for x in df_200['age'].values]

threshold_of_past_data=df_1000['age'].median()
df_1000['Z']=[get_indicator(x,threshold_of_past_data) for x in df_1000['age'].values]

##### a.

Lets define:

$ P(Z=1|Y=1)={p_1} $ 

$ P(Z=1|Y=0)={p_2} $

$ \eta(p)=log(p/1-p)$ 

$\psi = \eta(p1)-\eta(p2) = log((x_{00}*x_{11})/(x_{01}*x_{10}))$

In [7]:
# estimation of p1,p2
def estimate_p(df,category):
    indicators=df[df['income'] ==category]['Z'].values
    total=sum(indicators)
    p=total/len(indicators)
    return p

p1=estimate_p(df_200,1)
p2=estimate_p(df_200,0)

psi_a=math.log(p1/(1-p1))-math.log(p2/(1-p2))
print('Estimation of log OR is: {}'.format(psi_a))

# esimates stansart eror of log odds ratio using bootstrap
def estimate_se_via_bootstrap(df,psi,B,n=150):
    sampled_se_list=[]
    for i in range(B):
        df=df.sample(n, replace=True)
        df=df.reset_index(drop=True)
        x00=df[df['income'] ==0][df['Z'] ==0].shape[0]
        x10=df[df['income'] ==1][df['Z'] ==0].shape[0]
        x01=df[df['income'] ==0][df['Z'] ==1].shape[0]
        x11=df[df['income'] ==1][df['Z'] ==1].shape[0]
        se=1/x00 if x00!=0 else 0
        se=se+1/x10 if x10!=0 else se
        se=se+1/x01 if x01!=0 else se
        se=se+1/x11 if x11!=0 else se
        psi_se=(np.sqrt(se))*psi
        sampled_se_list.append(psi_se)
    return sum(sampled_se_list)/B


se = estimate_se_via_bootstrap(df_200,psi_a,B=500)


z_quantile=norm.ppf(1-0.05/2)
CI=[psi_a-z_quantile*se,psi_a+z_quantile*se]
    
print('Confidence Interval of log OR is {}'.format(CI))

Estimation of log OR is: 0.7693214114555615
Confidence Interval of log OR is [0.5795282192985598, 0.9591146036125633]


##### b.

<a href="https://ibb.co/c1mpj5T"><img src="https://i.ibb.co/c1mpj5T/2-b.png" alt="2-b" border="0"></a> 

(if you cant see the picture, please enter the link manually, to see the calculations.)

For standart uniform prior we know that MAP estimator of $p_1$ equal to MLE estimator.

From tutorial 6 we know MLE for $p_1$ is $\bar X_{11}/n$  and for $p_2$ is $\bar X_{01}/m$ 

$X_{01}$ ~ $Binomial(X_{0},p_2)$

$X_{11}$ ~ $Binomial(X_{1},p_1)$


In [8]:
p1_b=df_200[df_200['income'] ==1][df_200['Z'] ==1]['Z'].mean()/df_200[df_200['income'] ==1].shape[0]
p2_b=df_200[df_200['income'] ==0][df_200['Z'] ==1]['Z'].mean()/df_200[df_200['income'] ==0].shape[0]
psi_b=math.log(p1_b/(1-p1_b))-math.log(p2_b/(1-p2_b))
print('Estimation of log OR is: {}'.format(psi_b))

def get_credible_int_b(df,sample_size,y,z):
    a=df[df['income'] ==y][df['Z'] ==z]['Z'].sum()+1
    b=df[df['income'] ==y].shape[0]*df[df['income'] ==y][df['Z'] ==z].shape[0]-df[df['income'] ==y][df['Z'] ==z]['Z'].sum()+1
    samples=np.random.beta(a, b, sample_size)
    samples.sort()
    credible_int=[samples[int(0.05*sample_size/2)],samples[int((1-0.05/2)*sample_size)]]
    return credible_int
ci_p1=get_credible_int_b(df_200,100000,1,1)
ci_p2=get_credible_int_b(df_200,100000,0,1)
ci_psi=[math.log(ci_p1[0]/(1-ci_p1[0]))-math.log(ci_p2[0]/(1-ci_p2[0])),math.log(ci_p1[1]/(1-ci_p1[1]))-math.log(ci_p2[1]/(1-ci_p2[1]))]

print("Credible Interval of psi is {}".format(ci_psi))

Estimation of log OR is: 1.4053425560905852
Credible Interval of psi is [1.2487576215884806, 1.5634423426247914]


##### c.

Inserting jeffreys prior and solving MAP we get that estimator for $p_1$ is $(\Sigma^N X_{11} +0.5)/(1+Nn)$
and for $p_2$ is $(\Sigma^M X_{01}+0.5)/(1+Mm)$

<a href="https://ibb.co/jZ7HKnr"><img src="https://i.ibb.co/jZ7HKnr/2-c.png" alt="2-c" border="0"></a> 


(if you cant see the picture, please enter the link manually, to see the calculations.)

In [9]:
p1_c=(df_200[df_200['income'] ==1][df_200['Z'] ==1]['Z'].sum()+0.5)/(1+df_200[df_200['income'] ==1].shape[0]*df_200[df_200['income'] ==1][df_200['Z'] ==1].shape[0])
p2_c=(df_200[df_200['income'] ==0][df_200['Z'] ==1]['Z'].sum()+0.5)/(1+df_200[df_200['income'] ==0].shape[0]*df_200[df_200['income'] ==0][df_200['Z'] ==1].shape[0])
psi_c=math.log(p1_c/(1-p1_c))-math.log(p2_c/(1-p2_c))
print('Estimation of log OR is: {}'.format(psi_c))

def get_credible_int_c(df,sample_size,y,z):
    a=df[df['income'] ==y][df['Z'] ==z]['Z'].sum()+1.5
    b=df[df['income'] ==y].shape[0]*df[df['income'] ==y][df['Z'] ==z].shape[0]-df[df['income'] ==y][df['Z'] ==z]['Z'].sum()+1.5
    samples=np.random.beta(a, b, sample_size)
    samples.sort()
    credible_int=[samples[int(0.05*sample_size/2)],samples[int((1-0.05/2)*sample_size)]]
    return credible_int
ci_p1=get_credible_int_c(df_200,10000,1,1)
ci_p2=get_credible_int_c(df_200,10000,0,1)
ci_psi=[math.log(ci_p1[0]/(1-ci_p1[0]))-math.log(ci_p2[0]/(1-ci_p2[0])),math.log(ci_p1[1]/(1-ci_p1[1]))-math.log(ci_p2[1]/(1-ci_p2[1]))]

print("Credible Interval of psi is {}".format(ci_psi))

Estimation of log OR is: 1.4172062381878079
Credible Interval of psi is [1.259031984782486, 1.5727201174502246]


##### d.

<a href="https://ibb.co/qFfXndw"><img src="https://i.ibb.co/qFfXndw/2-d.png" alt="2-d" border="0"></a>

(if you cant see the picture, please enter the link manually, to see the calculations.)

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def get_betas_params(observations):
    a, b, loc, scale =stats.beta.fit(observations)
    return  (a,b)

a_p1,b_p1=get_betas_params(df_1000[df_1000['income'] ==1][df_1000['Z'] ==1]['Z'].values.tolist())
a_p2,b_p2=get_betas_params(df_1000[df_1000['income'] ==0][df_1000['Z'] ==1]['Z'].values.tolist())


p1_d=(df_200[df_200['income'] ==1][df_200['Z'] ==1]['Z'].sum()+a_p1)/(a_p1+b_p1+df_200[df_200['income'] ==1].shape[0]*df_200[df_200['income'] ==1][df_200['Z'] ==1].shape[0])
p2_d=(df_200[df_200['income'] ==0][df_200['Z'] ==1]['Z'].sum()+a_p2)/(a_p2+b_p2+df_200[df_200['income'] ==0].shape[0]*df_200[df_200['income'] ==0][df_200['Z'] ==1].shape[0])
psi_d=math.log(p1_d/(1-p1_d))-math.log(p2_d/(1-p2_d))
print('Estimation of log OR is: {}'.format(psi_d))

def get_credible_int_d(df,sample_size,y,z,alpha,beta):
    a=df[df['income'] ==y][df['Z'] ==z]['Z'].sum()+alpha
    b=df[df['income'] ==y].shape[0]*df[df['income'] ==y][df['Z'] ==z].shape[0]-df[df['income'] ==y][df['Z'] ==z]['Z'].sum()+beta
    samples=np.random.beta(a, b, sample_size)
    samples.sort()
    credible_int=[samples[int(0.05*sample_size/2)],samples[int((1-0.05/2)*sample_size)]]
    return credible_int

ci_p1=get_credible_int_b(df_200,10000,1,1)
ci_p2=get_credible_int_b(df_200,10000,0,1)
ci_psi=[math.log(ci_p1[0]/(1-ci_p1[0]))-math.log(ci_p2[0]/(1-ci_p2[0])),math.log(ci_p1[1]/(1-ci_p1[1]))-math.log(ci_p2[1]/(1-ci_p2[1]))]

print("Credible Interval of psi is {}".format(ci_psi))

Estimation of log OR is: 1.4291148347524092
Credible Interval of psi is [1.2511512653577856, 1.5626026699806865]


##### e.
All the estomators are quite simmilar.But, estimator from 2.a is less similar to the others.
We tend to think that estimator from 2.d is more accurate than the others.
Jaffrey's prior is better than the flat one and it is non informative.
Prior calculated via past samples, assumes knowlege about the disrtibution and seems to be more precise, because in retrospect all the data came from the same data set.

In [11]:
print('2.a Estimation of log OR is: {}'.format(psi_a))
print('2.b Estimation of log OR is: {}'.format(psi_b))
print('2.c Estimation of log OR is: {}'.format(psi_c))
print('2.d Estimation of log OR is: {}'.format(psi_d))

2.a Estimation of log OR is: 0.7693214114555615
2.b Estimation of log OR is: 1.4053425560905852
2.c Estimation of log OR is: 1.4172062381878079
2.d Estimation of log OR is: 1.4291148347524092


# Part 2: Missing Data

Are age, gender and education (edication_num) correlated with hours per week?
Y=hours per week

### 1.

In [13]:
df_1000=df_full.sample(n = 1000)
df_1000['aux_columns_of_ones']= [1]*df_1000.shape[0]
df_1000 = df_1000[['aux_columns_of_ones','gender','age','educational-num','hours-per-week']]

As you can see above, df_full does not consist any missing values, therefore, we'll get the required data.

### 2.

In [14]:
def create_targets_and_lables(df_data):
    X=df_data.iloc[:, 0:df_data.shape[1]-1].values
    Y=df_data.iloc[:, df_data.shape[1]-1:].values
    return X,Y

def estimate_beta(df_data):
    X,Y =create_targets_and_lables(df_data)
    beta_est=((np.linalg.pinv(X.T@X))@X.T)@Y
    return beta_est
  

X_sampled,Y_sampled=create_targets_and_lables(df_1000)
beta_est=estimate_beta(df_1000)

Y_pred = X_sampled@beta_est

n_p=X_sampled.shape[0]-X_sampled.shape[1]
mse_resid=np.sum(np.square(Y_sampled-Y_pred))/n_p
C = np.linalg.inv(X_sampled.T@X_sampled)
C *= mse_resid
SE = np.sqrt(C)


CI_on_cov_matrix={}
z_quantile=norm.ppf(1-0.05/2)
print('CI per beta:')

for i,beta in enumerate(beta_est):
    ci=[beta[0]-z_quantile*SE[i][i],beta[0]+z_quantile*SE[i][i]]
    CI_on_cov_matrix["beta_{}".format(i)]=ci
    print("beta{} is: {}, ci is: {}".format(i,beta[0],ci))
    


CI per beta:
beta0 is: 26.888053434943288, ci is: [23.239600629374543, 30.536506240512033]
beta1 is: 5.596043421944073, ci is: [4.130282598711968, 7.061804245176178]
beta2 is: 0.09007758503883385, ci is: [0.034287266220053375, 0.14586790385761433]
beta3 is: 0.6748538092353172, ci is: [0.3973435827563493, 0.9523640357142852]


### 3.

In [15]:
df_1000=df_1000.sort_values(by=['hours-per-week'])

random_ber_vars=[]
for i,sample in enumerate(df_1000['hours-per-week'].values):
    random_ber_vars.append(np.random.binomial(1, i/(i+2)))
    
df_1000['random_ber_var']= random_ber_vars  
# sample 500 samples with bernoulli == 1 and remove them.
df=df_1000.query("random_ber_var == 1").sample(n=500)
df.loc[df['random_ber_var'] == 1, 'hours-per-week'] = 'NaN'
df_1000.update(df)
df_1000.loc[df_1000['hours-per-week'] == 'NaN', 'hours-per-week'] = np.nan
df_1000

,aux_columns_of_ones,gender,age,educational-num,hours-per-week,random_ber_var
9846,1.0,0.0,24.0,12.0,3,0.0
46684,1.0,0.0,56.0,9.0,4,0.0
45507,1.0,0.0,17.0,6.0,5,0.0
42061,1.0,0.0,20.0,10.0,6,0.0
15795,1.0,1.0,59.0,14.0,7,0.0
...,...,...,...,...,...,...
25999,1.0,1.0,49.0,15.0,NaN,1.0
29205,1.0,1.0,37.0,13.0,90,1.0
44644,1.0,1.0,52.0,10.0,90,1.0
22351,1.0,1.0,48.0,9.0,NaN,1.0


### 4.

#### a.

In [16]:
df_1000 = df_1000[['aux_columns_of_ones','gender','age','educational-num','hours-per-week']]
df_no_nan=df_1000.dropna()
df_no_nan=df_no_nan[['aux_columns_of_ones','gender','age','educational-num','hours-per-week']]
X_sampled,Y_sampled=create_targets_and_lables(df_no_nan)
beta_est=estimate_beta(df_no_nan)

Y_pred = X_sampled@beta_est

n_p=X_sampled.shape[0]-X_sampled.shape[1]
mse_resid=np.sum(np.square(Y_sampled-Y_pred))/n_p
C = np.linalg.inv(X_sampled.T@X_sampled)
C *= mse_resid
SE = np.sqrt(C)


CI_on_cov_matrix={}
z_quantile=norm.ppf(1-0.05/2)
print('CI per beta:')

for i,beta in enumerate(beta_est):
    ci=[beta[0]-z_quantile*SE[i][i],beta[0]+z_quantile*SE[i][i]]
    CI_on_cov_matrix["beta_{}".format(i)]=ci
    if beta[0]< ci[1] and beta[0]> ci[0]:
        temp = 'in'
    else:
        temp ='not in'
    print("beta{} is: {}, ci is: {}, beta{} is {} the ci.".format(i,beta[0],ci,i, temp))

CI per beta:
beta0 is: 27.416655414288144, ci is: [22.035312495952894, 32.79799833262339], beta0 is in the ci.
beta1 is: 5.957272097087067, ci is: [3.8144928098215574, 8.100051384352577], beta1 is in the ci.
beta2 is: 0.12108698838470636, ci is: [0.03830507228676118, 0.20386890448265155], beta2 is in the ci.
beta3 is: 0.41224902825452064, ci is: [0.012353397010886558, 0.8121446594981547], beta3 is in the ci.


#### b.

In [17]:
# data wihout missing values. will be used in regression imputation
df_no_nan=df_1000.dropna()
df_no_nan=df_no_nan[['aux_columns_of_ones','gender','age','educational-num','hours-per-week']]
X_sampled,Y_sampled=create_targets_and_lables(df_no_nan)
beta_est=estimate_beta(df_no_nan)

# data with missing values.
df_with_nan=df_1000[df_1000.isnull().any(axis=1)]
df_with_nan=df_with_nan[['aux_columns_of_ones','gender','age','educational-num','hours-per-week']]
X_sampled_nan, Y_sampled_nan=create_targets_and_lables(df_with_nan)

# replace missing values using imputation
Y_imputation = X_sampled_nan@beta_est
df_b=df_with_nan.copy()
df_b['hours-per-week']=Y_imputation

# estimate beta using new data.
beta_est=estimate_beta(df_b)
Y_pred = X_sampled_nan@beta_est

n_p=X_sampled_nan.shape[0]-X_sampled_nan.shape[1]
mse_resid=np.sum(np.square(Y_imputation-Y_pred))/n_p
C = np.linalg.inv(X_sampled_nan.T@X_sampled_nan)
C *= mse_resid
SE = np.sqrt(C)


CI_on_cov_matrix={}
z_quantile=norm.ppf(1-0.05/2)
print('CI per beta:')


for i,beta in enumerate(beta_est):
    ci=[beta[0]-z_quantile*SE[i][i],beta[0]+z_quantile*SE[i][i]]
    CI_on_cov_matrix["beta_{}".format(i)]=ci
    if beta[0]< ci[1] and beta[0]> ci[0]:
        temp = 'in'
    else:
        temp ='not in'
    print("beta{} is: {}, ci is: {}, beta{} is {} the ci.".format(i,beta[0],ci,i, temp))

CI per beta:
beta0 is: 27.416655414286602, ci is: [27.41665541428645, 27.416655414286755], beta0 is in the ci.
beta1 is: 5.957272097087174, ci is: [5.957272097087113, 5.957272097087236], beta1 is in the ci.
beta2 is: 0.12108698838472734, ci is: [0.12108698838472505, 0.12108698838472963], beta2 is in the ci.
beta3 is: 0.41224902825457865, ci is: [0.4122490282545669, 0.4122490282545904], beta3 is in the ci.


As we can see, we got the same results as we got in section a - all betas are in their ci.

#### c.

In [18]:
# data wihout missing values. will be used in regression imputation
df_no_nan=df_1000.dropna()
df_no_nan=df_no_nan[['aux_columns_of_ones','gender','age','educational-num','hours-per-week']]
X_sampled,Y_sampled=create_targets_and_lables(df_no_nan)
beta_est=estimate_beta(df_no_nan)

# data with missing values.
df_with_nan=df_1000[df_1000.isnull().any(axis=1)]
df_with_nan=df_with_nan[['aux_columns_of_ones','gender','age','educational-num','hours-per-week']]
X_sampled_nan, Y_sampled_nan=create_targets_and_lables(df_with_nan)


def multiple_imputations(m,df,df_no_nan,df_with_nan):
    imputations=[]
    X_sampled,_=create_targets_and_lables(df)
    mu, sigma = norm.fit(X_sampled)
    X_sampled_nan, _=create_targets_and_lables(df_with_nan)

    betas_estimations=[]
    for i in range(0,m):
        # replace missing values using imputation for each i- Step 1
        Y_imputation_i = np.random.normal(mu, sigma, 500)
        df_i=df_with_nan.copy()
        df_i['hours-per-week']=Y_imputation
        df_i=pd.concat([df_i,df_no_nan])
        # estimate beta using new data for each i- Step 2
        beta_est=estimate_beta(df_i)
        betas_estimations.append(beta_est)
        Y_pred = X_sampled_nan@beta_est
        imputations.append(Y_pred)
#         return avg. of imputations,avg. of betas (estamotion of betas-Step 3), betas 
    return (np.mean(imputations, axis=0),np.mean(betas_estimations, axis=0),betas_estimations)


Y_pred,betas,betas_estimations=multiple_imputations(100,df_1000,df_no_nan,df_with_nan)

n_p=X_sampled_nan.shape[0]-X_sampled_nan.shape[1]
mse_resid=np.sum(np.square(Y_imputation-Y_pred))/n_p
C = np.linalg.inv(X_sampled_nan.T@X_sampled_nan)
C *= mse_resid
SE = np.sqrt(C)


CI_on_cov_matrix={}
z_quantile=norm.ppf(1-0.05/2)
print('CI per beta:')

for i,beta in enumerate(betas):
    ci=[beta[0]-z_quantile*SE[i][i],beta[0]+z_quantile*SE[i][i]]
    CI_on_cov_matrix["beta_{}".format(i)]=ci
    if beta[0]< ci[1] and beta[0]> ci[0]:
        temp = 'in'
    else:
        temp ='not in'
    print("beta{} is: {}, ci is: {}, beta{} is {} the ci.".format(i,beta[0],ci,i, temp))

CI per beta:
beta0 is: 27.416655414283603, ci is: [27.416655414283202, 27.416655414284005], beta0 is in the ci.
beta1 is: 5.957272097087537, ci is: [5.957272097087373, 5.9572720970877], beta1 is in the ci.
beta2 is: 0.12108698838474666, ci is: [0.12108698838474054, 0.12108698838475278], beta2 is in the ci.
beta3 is: 0.4122490282547587, ci is: [0.41224902825472726, 0.4122490282547901], beta3 is in the ci.


#### d.

In [19]:
def within_imputation_variance(imputations):
    summations=0
    M=len(imputations)
    for imputation in imputations:
        summation=+imputation.var()
    
    return summation/M


def between_imputation_variance(Y_pred,imputations):
    summations=[]*len(beta_est)
    M=len(imputations)
    for imputation in imputations:
        diff=imputation-Y_pred
        summations=+(imputation-Y_pred)**2
    flat_list =np.asarray([val for sublist in summations for val in sublist])
    return (M+1)*flat_list/(M*(M-1))

se_est=np.sqrt(within_imputation_variance(betas_estimations)+between_imputation_variance(betas,betas_estimations))
print("Estimation of s.e of each beta using Rubun's formula: {}".format(se_est))
print()

z_quantile=norm.ppf(1-0.05/2)
for i,beta in enumerate(betas):
    ci=[beta[0]-z_quantile*se,beta[0]+z_quantile*se]
    CI_on_cov_matrix["beta_{}".format(i)]=ci
    if beta[0]< ci[1] and beta[0]> ci[0]:
        temp = 'in'
    else:
        temp ='not in'
    print("beta{} is: {}, ci is: {}, beta{} is {} the ci.".format(i,beta[0],ci,i, temp))

Estimation of s.e of each beta using Rubun's formula: [1.11794573 1.11794573 1.11794573 1.11794573]

beta0 is: 27.416655414283603, ci is: [27.226862222126602, 27.606448606440605], beta0 is in the ci.
beta1 is: 5.957272097087537, ci is: [5.767478904930535, 6.147065289244539], beta1 is in the ci.
beta2 is: 0.12108698838474666, ci is: [-0.06870620377225509, 0.3108801805417484], beta2 is in the ci.
beta3 is: 0.4122490282547587, ci is: [0.22245583609775693, 0.6020422204117604], beta3 is in the ci.


#### e.
As we know:
${R|X_1,⋯, X_k} ∼ Ber(e^{X^T\beta}/(1 + e^{X^T\beta})) $

Using logistic regression:

In [76]:
from sklearn.linear_model import LogisticRegression
one = np.ones(df_no_nan.shape[0])
X_sampled,Y_sampled=create_targets_and_lables(df_no_nan)
log_reg = LogisticRegression(random_state=0)
log_reg.fit(X_sampled, Y_sampled.astype('int'))
beta_log_hat = log_reg.coef_

beta_log_hat=np.mean(beta_log_hat, axis=0)


#### f.

In class we seen that IPW estimator is: $ 1/n {\Sigma}^{n}_{i=1}R_i/\hat{\pi}(W_i)\cdot s(Y_i;\theta) $
We'll replace s(Y_i;\theta) with $ ||{X^T \beta - y}|| $

In [90]:
def calculate_pi(X,beta):
    pi=np.exp(X@beta)/(1+np.exp(X@beta))
    return pi

In [102]:
X_sampled,Y_sampled=create_targets_and_lables(df_no_nan)
pi_w = calculate_pi(X_sampled,beta_log_hat)
pi_w = np.diag(pi_w)
print(Y_sampled.shape)
beta_ipw = np.linalg.lstsq(pi_w@X_sampled, pi_w@Y_sampled)
beta_ipw[0]

(500, 1)


TypeError: No loop matching the specified signature and casting was found for ufunc lstsq_n

#### g.

#### h.